# **2nd Approach:** Tensorflow Neural Network

### Let's develop a Baseline Neural Network Model using a simple model that has:
- A single fully connected hidden layer with the same number of neurons (i.e. 27) as input attributes
- A ReLU activation function
- A batch normalization layer between two dense layers
- Adaptive Moment Estimation (Adam) momentum optimizer

In [12]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from time import time
import pandas as pd
import numpy as np

import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential 
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test
%store -r ind_vars
%store -r dep_var
%store -r X
%store -r y
%store -r df

In [13]:
tf.random.set_seed(45)

#define baseline model
model = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=(30,), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', optimizer= tf.keras.optimizers.Adam(0.02))

#fit model
history = model.fit(X_train, y_train, epochs= 300)
model.summary()

#test model
print(model.predict(X_test))

Epoch 1/300
347/347 [==============================] - 1s 1ms/step - loss: 1751061248.0000
Epoch 2/300
347/347 [==============================] - 0s 982us/step - loss: 1197266816.0000
Epoch 3/300
347/347 [==============================] - 0s 1ms/step - loss: 578411456.0000
Epoch 4/300
347/347 [==============================] - 0s 1ms/step - loss: 251984880.0000
Epoch 5/300
347/347 [==============================] - 1s 2ms/step - loss: 136592208.0000
Epoch 6/300
347/347 [==============================] - 0s 1ms/step - loss: 107722320.0000
Epoch 7/300
347/347 [==============================] - 0s 1ms/step - loss: 94507920.0000
Epoch 8/300
347/347 [==============================] - 0s 1ms/step - loss: 74968208.0000
Epoch 9/300
347/347 [==============================] - 0s 1ms/step - loss: 66847996.0000
Epoch 10/300
347/347 [==============================] - 0s 1ms/step - loss: 68220440.0000
Epoch 11/300
347/347 [==============================] - 1s 1ms/step - loss: 70570104.0000
Epoch 12/

In [14]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


## Automatic Hyperparamater Tuning

In [15]:
def model_builder(hp):
  # Initialize the Sequential API and start stacking the layers
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=30, max_value=30)
  model = keras.models.Sequential([
      keras.layers.Dense(input_shape=(30,), units=hp_units, activation='relu'),
      # Add next layers
      keras.layers.BatchNormalization(),
      keras.layers.Dense(1)
  ])
      # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[2e-1, 2e-2, 2e-3, 2e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error',
                metrics=['accuracy'])
  return model

In [16]:
tf.random.set_seed(45)

# Instantiate the tuner
tuner = kt.Hyperband(model_builder, # the hypermodel
                     objective='val_accuracy', # objective to optimize
max_epochs=300,
)

# Perform hypertuning
tuner.search(X_train, y_train, epochs=300, validation_split=0.2)
best_hp=tuner.get_best_hyperparameters()[0]

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [17]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hp)

In [18]:
h_model.fit(X_train, y_train, epochs=300)
h_model.summary()
print(h_model.predict(X_test))

Epoch 1/300
347/347 [==============================] - 1s 1ms/step - loss: 1858345088.0000 - accuracy: 0.0000e+00
Epoch 2/300
347/347 [==============================] - 0s 1ms/step - loss: 1858015488.0000 - accuracy: 0.0000e+00
Epoch 3/300
347/347 [==============================] - 0s 1ms/step - loss: 1857639680.0000 - accuracy: 0.0000e+00
Epoch 4/300
347/347 [==============================] - 0s 1ms/step - loss: 1857226496.0000 - accuracy: 0.0000e+00
Epoch 5/300
347/347 [==============================] - 0s 1ms/step - loss: 1856762752.0000 - accuracy: 0.0000e+00
Epoch 6/300
347/347 [==============================] - 0s 1ms/step - loss: 1856273280.0000 - accuracy: 0.0000e+00
Epoch 7/300
347/347 [==============================] - 0s 1ms/step - loss: 1855737984.0000 - accuracy: 0.0000e+00
Epoch 8/300
347/347 [==============================] - 0s 1ms/step - loss: 1855111424.0000 - accuracy: 0.0000e+00
Epoch 9/300
347/347 [==============================] - 0s 1ms/step - loss: 1854476416.00

## Hyperparameter Tuning 
Some experiments and parameter hypertuning that were done to try and get better results were:
* Making the model Deeper by adding another layer, 2 layers instead of 1
* Making the model wider by adding more neurons, adding up to 128 in the first layer
* Changing the Adam optimizer to rmsprop. Increasing and Decreasing the Adam optimizer
* Increasing the training time from 300 to 600 epoch

All of these experiments did not lead to better results. These experiments were all also done using the preprocessed data and still do not give better results. So the baseline model with preprocessed data was chosen to be the Main Model. 

### Model 1

In [19]:
# Hyperparameter Tuning
# importing matplotlib module 
import matplotlib.pyplot as plt

X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

#build model 1
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', optimizer='adam')

#fit model
history = model.fit(X_train, y_train, epochs= 100, validation_data=(x_val, y_val))

model.summary()


#evaluate the model
print(model.predict(X_test))

Epoch 1/100
330/330 [==============================] - 2s 3ms/step - loss: 1858772608.0000 - val_loss: 1816252160.0000
Epoch 2/100
330/330 [==============================] - 1s 2ms/step - loss: 1853295232.0000 - val_loss: 1808162304.0000
Epoch 3/100
330/330 [==============================] - 1s 2ms/step - loss: 1844590080.0000 - val_loss: 1797342976.0000
Epoch 4/100
330/330 [==============================] - 1s 3ms/step - loss: 1832789504.0000 - val_loss: 1782532224.0000
Epoch 5/100
330/330 [==============================] - 1s 2ms/step - loss: 1818104704.0000 - val_loss: 1765772672.0000
Epoch 6/100
330/330 [==============================] - 1s 2ms/step - loss: 1800676608.0000 - val_loss: 1745784576.0000
Epoch 7/100
330/330 [==============================] - 1s 2ms/step - loss: 1780979584.0000 - val_loss: 1728213632.0000
Epoch 8/100
330/330 [==============================] - 1s 2ms/step - loss: 1759193600.0000 - val_loss: 1705379072.0000
Epoch 9/100
330/330 [===========================

In [20]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


### Model 2

In [21]:
X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

#build model 2
model = keras.models.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', optimizer='adam')

#fit model
history = model.fit(X_train, y_train, epochs= 100, validation_data=(x_val, y_val))

model.summary()

#evaluate the model
print(model.predict(X_test))

Epoch 1/100
313/313 [==============================] - 1s 2ms/step - loss: 1868411008.0000 - val_loss: 1676204032.0000
Epoch 2/100
313/313 [==============================] - 1s 2ms/step - loss: 1863481344.0000 - val_loss: 1669443072.0000
Epoch 3/100
313/313 [==============================] - 1s 2ms/step - loss: 1855659648.0000 - val_loss: 1659610240.0000
Epoch 4/100
313/313 [==============================] - 1s 2ms/step - loss: 1845029248.0000 - val_loss: 1648315776.0000
Epoch 5/100
313/313 [==============================] - 1s 2ms/step - loss: 1831660416.0000 - val_loss: 1635614592.0000
Epoch 6/100
313/313 [==============================] - 1s 2ms/step - loss: 1815784704.0000 - val_loss: 1619479296.0000
Epoch 7/100
313/313 [==============================] - 1s 2ms/step - loss: 1798028544.0000 - val_loss: 1604740736.0000
Epoch 8/100
313/313 [==============================] - 1s 2ms/step - loss: 1777937280.0000 - val_loss: 1583490048.0000
Epoch 9/100
313/313 [===========================

In [22]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


### Model 3

In [23]:
#build model 3
model = keras.models.Sequential([
    keras.layers.Dense(128, activation="tanh"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="tanh"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', 
              optimizer='adam')

#fit model
history = model.fit(X_train, 
                    y_train, 
                    epochs= 100, 
                    validation_data=(x_val, y_val))

model.summary()

#evaluate the model
print(model.predict(X_test))

Epoch 1/100
313/313 [==============================] - 1s 2ms/step - loss: 1868375552.0000 - val_loss: 1675928960.0000
Epoch 2/100
313/313 [==============================] - 0s 1ms/step - loss: 1863719424.0000 - val_loss: 1668081024.0000
Epoch 3/100
313/313 [==============================] - 0s 1ms/step - loss: 1856113408.0000 - val_loss: 1658908800.0000
Epoch 4/100
313/313 [==============================] - 0s 1ms/step - loss: 1845645440.0000 - val_loss: 1648431616.0000
Epoch 5/100
313/313 [==============================] - 0s 1ms/step - loss: 1832390144.0000 - val_loss: 1631388544.0000
Epoch 6/100
313/313 [==============================] - 0s 1ms/step - loss: 1816586496.0000 - val_loss: 1618997120.0000
Epoch 7/100
313/313 [==============================] - 0s 1ms/step - loss: 1798952576.0000 - val_loss: 1600019584.0000
Epoch 8/100
313/313 [==============================] - 1s 2ms/step - loss: 1778950144.0000 - val_loss: 1582179712.0000
Epoch 9/100
313/313 [===========================

In [24]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


### Model 4

In [25]:
#build model 4
model = keras.models.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', 
              optimizer='rmsprop')

#fit model
history = model.fit(X_train, 
                    y_train, 
                    epochs= 100, 
                    validation_data=(x_val, y_val))

model.summary()

#evaluate the model
print(model.predict(X_test))

Epoch 1/100
313/313 [==============================] - 2s 2ms/step - loss: 1868492288.0000 - val_loss: 1676323456.0000
Epoch 2/100
313/313 [==============================] - 0s 1ms/step - loss: 1865149824.0000 - val_loss: 1672138880.0000
Epoch 3/100
313/313 [==============================] - 0s 1ms/step - loss: 1860470016.0000 - val_loss: 1665696000.0000
Epoch 4/100
313/313 [==============================] - 0s 1ms/step - loss: 1854497280.0000 - val_loss: 1659348352.0000
Epoch 5/100
313/313 [==============================] - 0s 1ms/step - loss: 1846977408.0000 - val_loss: 1654097152.0000
Epoch 6/100
313/313 [==============================] - 0s 1ms/step - loss: 1838067712.0000 - val_loss: 1644517760.0000
Epoch 7/100
313/313 [==============================] - 0s 1ms/step - loss: 1828039424.0000 - val_loss: 1634961920.0000
Epoch 8/100
313/313 [==============================] - 0s 1ms/step - loss: 1816367360.0000 - val_loss: 1622043520.0000
Epoch 9/100
313/313 [===========================

In [26]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


### Model 5 (Main Model)

In [27]:
# MAIN MODEL 
# evaluate model with standardized dataset
from sklearn import preprocessing
# preprocess the data
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.reshape(-1,1)
y_test = y_test


tf.random.set_seed(42)

#define baseline model
model = keras.models.Sequential([
    keras.layers.Dense(units= 30, 
                       input_shape= (30,), 
                       activation= "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', 
              optimizer= tf.keras.optimizers.Adam(0.02))

#fit model
history = model.fit(X_train, 
                    y_train, 
                    epochs= 300)

model.summary()

#test model
print(model.predict(X_test))


Epoch 1/300
313/313 [==============================] - 1s 1ms/step - loss: 1765815552.0000
Epoch 2/300
313/313 [==============================] - 0s 1ms/step - loss: 1226368384.0000
Epoch 3/300
313/313 [==============================] - 0s 1ms/step - loss: 586272192.0000
Epoch 4/300
313/313 [==============================] - 0s 1ms/step - loss: 225612992.0000
Epoch 5/300
313/313 [==============================] - 0s 1000us/step - loss: 129806144.0000
Epoch 6/300
313/313 [==============================] - 0s 1ms/step - loss: 96830472.0000
Epoch 7/300
313/313 [==============================] - 0s 1ms/step - loss: 78311072.0000
Epoch 8/300
313/313 [==============================] - 0s 1ms/step - loss: 74567400.0000
Epoch 9/300
313/313 [==============================] - 1s 2ms/step - loss: 75815808.0000
Epoch 10/300
313/313 [==============================] - 0s 1ms/step - loss: 74509368.0000
Epoch 11/300
313/313 [==============================] - 0s 1ms/step - loss: 72142768.0000
Epoch 12/

In [28]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]


### Model 6

In [29]:
#define model 6 
model = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=(30,), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer= tf.keras.optimizers.Adam(0.01))

#fit model
history = model.fit(X_train, 
                    y_train, 
                    epochs= 300)

model.summary()

#test model
print(model.predict(X_test))

Epoch 1/300
313/313 [==============================] - 1s 1ms/step - loss: 1841843328.0000
Epoch 2/300
313/313 [==============================] - 0s 1ms/step - loss: 1680333952.0000
Epoch 3/300
313/313 [==============================] - 0s 1ms/step - loss: 1399853568.0000
Epoch 4/300
313/313 [==============================] - 0s 1ms/step - loss: 1066661312.0000
Epoch 5/300
313/313 [==============================] - 0s 1ms/step - loss: 753653056.0000
Epoch 6/300
313/313 [==============================] - 0s 1ms/step - loss: 491911584.0000
Epoch 7/300
313/313 [==============================] - 0s 955us/step - loss: 305711744.0000
Epoch 8/300
313/313 [==============================] - 0s 1ms/step - loss: 195472720.0000
Epoch 9/300
313/313 [==============================] - 0s 1ms/step - loss: 145372464.0000
Epoch 10/300
313/313 [==============================] - 0s 1ms/step - loss: 119306280.0000
Epoch 11/300
313/313 [==============================] - 0s 996us/step - loss: 108211864.0000


In [30]:
print(y_test)

[[25005.]
 [ 9664.]
 [ 6053.]
 ...
 [12549.]
 [ 1204.]
 [13000.]]
